In [1]:
!git clone https://github.com/jaywonchung/BERT4Rec-VAE-Pytorch  # clone repo

Cloning into 'BERT4Rec-VAE-Pytorch'...
remote: Enumerating objects: 166, done.
remote: Total 166 (delta 0), reused 0 (delta 0), pack-reused 166
Receiving objects: 100% (166/166), 3.05 MiB | 14.65 MiB/s, done.
Resolving deltas: 100% (61/61), done.


In [2]:
%cd BERT4Rec-VAE-Pytorch/


/content/BERT4Rec-VAE-Pytorch


While the requirements are installing, upload either the ratings or the preprocessed pickle files.
If you are doing inference, also upload the new template and main files.

#Make sure you restart runtime after you change out the files!

In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 52 kB 991 kB/s 
     |████████████████████████████████| 17.3 MB 104 kB/s 
     |████████████████████████████████| 773.1 MB 12 kB/s 
     |████████████████████████████████| 3.8 MB 36.1 MB/s 
     |████████████████████████████████| 10.4 MB 22.2 MB/s 
     |████████████████████████████████| 25.2 MB 1.7 MB/s 
     |████████████████████████████████| 829 kB 38.0 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=f58d5b427782129c73ab268ac78a1bef623166a4b7c0586fe4f97a7d3f656549
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=0c784c6609ca05c1c92a8baf838e14e68078d436b225f320c8df96eeefee7ade
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built wget future
  Attempting uninstall: numpy
    Found existing instal

**RUN FROM HERE TO TRAIN NEW MODEL**
You may want to run this once to create the apropriate data folders

In [ ]:
!python main.py --template train_bert

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
Input 1 for ml-1m, 20 for ml-20m: 1
0
Folder created: /content/BERT4Rec-VAE-Pytorch/experiments/test_2022-01-11_6
{'anneal_cap': 0.2,
 'bert_dropout': 0.1,
 'bert_hidden_units': 256,
 'bert_mask_prob': 0.15,
 'bert_max_len': 100,
 'bert_num_blocks': 2,
 'bert_num_heads': 4,
 'best_metric': 'NDCG@10',
 'dae_dropout': 0.5,
 'dae_hidden_dim': 600,
 'dae_latent_dim': 200,
 'dae_num_hidden': 0,
 'dataloader_code': 'bert',
 'dataloader_random_seed': 0.0,
 'dataset_code': 'ml-1m',
 'dataset_split_seed': 98765,
 'decay_step': 25,
 'device': 'cuda',
 'device_idx': '0',
 'enable_lr_schedule': True,
 'eval_set_size': 500,
 'experiment_description': 'test',
 'experiment_dir': 'experiments',
 'find_best_beta': False,
 'gamma': 1.0,
 'log_period_as_iter': 12800,
 'lr': 0.001,
 'metr

**RUN FROM HERE IF YOU WANT TO USE PRETRAINED MODEL**

In [1]:
%cd BERT4Rec-VAE-Pytorch/

/content/BERT4Rec-VAE-Pytorch


In [2]:
import sys
sys.argv = ['--template train_bert']
from main import *
from templates import *
# import subprocess
# proc = subprocess.Popen(['python', 'main.py',  '--template train_bert', '--mode train'])
print(sys.argv)
args.template = "test_bert"
set_template(args)
model,test_loader=train()

import torch

#from conversion import *
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
import pickle
import pandas as pd
from types import SimpleNamespace


# Load best model file
export_root = '/content/BERT4Rec-VAE-Pytorch/best_model'
best_model = torch.load(os.path.join(export_root, 'models', 'best_acc_model.pth')).get('model_state_dict')
print (list(best_model.keys()))
# for key in list(best_model.keys()):
#     best_model[key.replace('bert.', '')] = best_model.pop(key)


model.load_state_dict(best_model)
model.to("cuda")
model.eval()

# Get sample tensor to load data into the model
dataset = test_loader.dataset
trainset = DataLoader(dataset=dataset,
                      batch_size=1,
                      shuffle=False,
                      pin_memory=True)
data = tqdm(trainset)
ids_and_tensors = list(enumerate(data))

# Load dataset (to get item mapping)
export_root = '/content/BERT4Rec-VAE-Pytorch/Data/preprocessed/ml-1m_min_rating0-min_uc5-min_sc0-splitleave_one_out/dataset.pkl'
data = pickle.load(open(export_root, "rb"))
train = data['train']
val = data['val']
test = data['test']
umap = data['umap']
smap = data['smap']
user_count = len(umap)
item_count = len(smap)
inv_map = {v: k for k, v in smap.items()}
inv_user_map = {v: k for k, v in umap.items()}
user_count, item_count

# Function to get top predictions for a list of mal iDs
def predict(input_movies, k=100):
    user_id = 1111 # doesnt matter
    source_movies = input_movies

    with torch.no_grad():
        movies=[]
        for id in input_movies:
          try: 
            movies.append(smap[id])

          except KeyError:
            print(id, "is not an accepted input anime, ignoring")
        if movies==[]:
          print("No valid anime, halting prediction")
          return
        start = len(ids_and_tensors[user_id][1][0][0]) - len(movies)
        for x in range(0, len(ids_and_tensors[user_id][1][0][0])):
            ids_and_tensors[user_id][1][0][0][x] = 0
        for x in range(start, len(ids_and_tensors[user_id][1][0][0])):
            ids_and_tensors[user_id][1][0][0][x] = movies[x-start]
        scores = model(ids_and_tensors[user_id][1][0].to('cuda'))
        scores = scores[:, -1, :]
        predictions = torch.topk(scores, k=k)
        tops = predictions[1].cpu().numpy()[0]

    output_movies = []
    for x in range(0, len(tops)):
        output_movies.append(inv_map[tops[x]])

    final_results = [movie for movie in output_movies if movie not in source_movies]
    source, prediction = source_movies, final_results

    return source, prediction

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


['--template train_bert']
Input 1 for ml-1m, 20 for ml-20m: 1
0
Folder created: /content/BERT4Rec-VAE-Pytorch/experiments/test_2022-01-11_3
{'anneal_cap': 0.2,
 'bert_dropout': 0.1,
 'bert_hidden_units': 256,
 'bert_mask_prob': 0.15,
 'bert_max_len': 100,
 'bert_num_blocks': 2,
 'bert_num_heads': 4,
 'best_metric': 'NDCG@10',
 'dae_dropout': 0.5,
 'dae_hidden_dim': 600,
 'dae_latent_dim': 200,
 'dae_num_hidden': 0,
 'dataloader_code': 'bert',
 'dataloader_random_seed': 0.0,
 'dataset_code': 'ml-1m',
 'dataset_split_seed': 98765,
 'decay_step': 25,
 'device': 'cuda',
 'device_idx': '0',
 'enable_lr_schedule': True,
 'eval_set_size': 500,
 'experiment_description': 'test',
 'experiment_dir': 'experiments',
 'find_best_beta': False,
 'gamma': 1.0,
 'log_period_as_iter': 12800,
 'lr': 0.001,
 'metric_ks': [1,
               5,
               10,
               20,
               50,
               100],
 'min_rating': 0,
 'min_sc': 0,
 'min_uc': 5,
 'mode': 'train',
 'model_code': 'bert',


  0%|          | 364/234882 [00:00<01:04, 3629.67it/s]

['bert.embedding.token.weight', 'bert.embedding.position.pe.weight', 'bert.transformer_blocks.0.attention.linear_layers.0.weight', 'bert.transformer_blocks.0.attention.linear_layers.0.bias', 'bert.transformer_blocks.0.attention.linear_layers.1.weight', 'bert.transformer_blocks.0.attention.linear_layers.1.bias', 'bert.transformer_blocks.0.attention.linear_layers.2.weight', 'bert.transformer_blocks.0.attention.linear_layers.2.bias', 'bert.transformer_blocks.0.attention.output_linear.weight', 'bert.transformer_blocks.0.attention.output_linear.bias', 'bert.transformer_blocks.0.feed_forward.w_1.weight', 'bert.transformer_blocks.0.feed_forward.w_1.bias', 'bert.transformer_blocks.0.feed_forward.w_2.weight', 'bert.transformer_blocks.0.feed_forward.w_2.bias', 'bert.transformer_blocks.0.input_sublayer.norm.a_2', 'bert.transformer_blocks.0.input_sublayer.norm.b_2', 'bert.transformer_blocks.0.output_sublayer.norm.a_2', 'bert.transformer_blocks.0.output_sublayer.norm.b_2', 'bert.transformer_blocks.

100%|██████████| 234882/234882 [01:00<00:00, 3863.80it/s]


In [11]:
#Print top 10 recommendations
x,y = predict([22319,31240,30831,19815])
# Above anime in order^:
#  tokyo ghoul, re zero, konosuba, no game no life
df=pd.DataFrame(y)
df[:10]

,0
0,32937
1,31043
2,15809
3,30276
4,21881
5,22199
6,18679
7,20507
8,10793
9,23273


In [12]:
counter=0
for x in df.itertuples():
  if counter == 10:
    break
  counter+=1
  print("https://myanimelist.net/anime/"+str(x[1]))

https://myanimelist.net/anime/32937
https://myanimelist.net/anime/31043
https://myanimelist.net/anime/15809
https://myanimelist.net/anime/30276
https://myanimelist.net/anime/21881
https://myanimelist.net/anime/22199
https://myanimelist.net/anime/18679
https://myanimelist.net/anime/20507
https://myanimelist.net/anime/10793
https://myanimelist.net/anime/23273
